In [1]:
from map_utils import *
from LoadLSeg import *

/home/fyp/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Namespace(model='encnet', backbone='clip_vitl16_384', dataset='fss', workers=16, base_size=520, crop_size=480, train_split='train', aux=False, se_loss=False, se_weight=0.2, batch_size=16, test_batch_size=16, no_cuda=False, seed=1, weights='/home/fyp/lang-seg/checkpoints/demo_e200.ckpt', eval=False, acc_bn=False, test_val=False, no_val=False, module='clipseg_DPT_test_v2', scale_inv=False, widehead=True, widehead_hr=False, ignore_index=255, jobname='default', strict=True, use_pretrained='True', arch_option=0, nshot=0, fold=0, nworker=0, bsz=1, benchmark='pascal', datapath='data/Datasets_HSN', activation='relu', cuda=True)
Loading model...


/home/fyp/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint checkpoints/demo_e200.ckpt`
/home/fyp/lang-seg/modules/lsegmentation_module_zs.py:50: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = amp.GradScaler(enabled=self.enabled)

:=========== Few-shot Seg. with HSNet ===========
|             datapath: data/Datasets_HSN       
|       

use norm: [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
['ac_wall', 'acorn', 'adhensive_tape', 'adidas_logo1', 'adidas_logo2', 'afghan_hound', 'african_elephant', 'african_grey', 'agama', 'air_strip', 'aircraft_carrier', 'airedale', 'airship', 'almond', 'ambulance', 'american_staffordshire', 'anemone_fish', 'angora', 'apple', 'armour', 'ashtray', 'assult_rifle', 'aubergine', 'avocado', 'baboon', 'backpack', 'bagel', 'balance_weight', 'bald_eagle', 'ballpoint', 'banana', 'band-aid', 'banded_gecko', 'barometer', 'baseball_bat', 'baseball_player', 'basketball', 'bassoon', 'bathtub', 'battery', 'beacon', 'beaker', 'beam_bridge', 'bear', 'beaver', 'bedlington_terrier', 'bee_house', 'besom', 'birdhouse', 'bison', 'black_grouse', 'black_stork', 'black_swan', 'blossom_card', 'boa_constrictor', 'bolotie', 'bomb', 'border_terrier', 'boston_bull', 'bottle_cap', 'bouzouki', 'box_turtle', 'bra', 'bracelet', 'bradypod', 'brain_coral', 'brambling', 'brasscica', 'brick', 'brick_card', 'brick_tea', 'briefcase', 'br

In [2]:
sfpc = ServerFeaturePointCloudMap()

In [3]:
# read a image
from PIL import Image
image = Image.open("/home/fyp/Pictures/TestSeg/office1.jpg")

In [4]:
sfpc.set_model(model)

In [5]:
sfpc.pil_image = image
sfpc.update_feature()

In [6]:
feature = sfpc.current_feature
def PCA_cp(feat, n_components=20):
    '''
    Applies PCA to reduce the dimensionality of the feature map.

    Args:
        feat (torch.Tensor): The input feature map.
        n_components (int): The number of principal components to keep.

    Returns:
        numpy.ndarray: The transformed feature map with reduced dimensionality.
    '''
    pca = PCA(n_components=n_components)
    feat_map = feat.flatten(start_dim=0, end_dim=1).detach().cpu().numpy()
    features = pca.fit_transform(feat_map)
    return features

In [7]:
from cuml.decomposition import PCA as cuPCA

In [8]:
features = sfpc.fip.PCA_cuda(sfpc.current_feature)

In [9]:
clustered_image = sfpc.fip.Cluster_cuda(features)

In [10]:
key_pixels = sfpc.fip.obtain_key_pixels(sfpc.current_feature, clustered_image)

In [11]:
type(key_pixels)

list

In [13]:
type(key_pixels[0])

tuple

In [17]:
print(type(key_pixels[0][1][1]))

<class 'numpy.ndarray'>
